In [1]:
from utility import EnglishTextProcessor
import numpy as np
from tqdm import tqdm
import json

In [2]:
from gensim.models import FastText, Word2Vec

In [4]:
def load_json(filename):
    d = None
    with open(filename) as file:
        d = json.load(file)
    
    return d

In [5]:
etp = EnglishTextProcessor()
author = load_json('author_filtered_citations.json')

In [6]:
import json
import os
import pickle

In [8]:
def iter_batches(abstract_folder):
    json_file_names = sorted(os.listdir(abstract_folder), key=lambda x: int(x.split('_')[-1].split('.')[0]))
    for file_name in json_file_names:
        file_path = os.path.join(abstract_folder, file_name)
        try:
            id_to_abstract_dict = None
            with open(file_path) as id_to_abstract_json_file:
                id_to_abstract_dict = json.load(id_to_abstract_json_file)

            yield id_to_abstract_dict

        except ValueError:
            continue

In [7]:
from aminer.recall.query_es import get_abstract_by_pids, get_references_by_pid

test_recs = {}
for id in author:
    test_recs[id] = get_references_by_pid(id)

In [8]:
relevant_ids = set(test_recs.keys())
for id in test_recs:
    relevant_ids = relevant_ids.union(test_recs[id])

In [15]:
relevant_id_keywords = {id:set() for id in relevant_ids}
for i in tqdm(range(411)):
    batch = load_json('../support/keyword_batches/{}.json'.format(i))
    for id in batch:
        if id in relevant_id_keywords:
            relevant_id_keywords[id] = batch[id]

100%|██████████| 411/411 [00:42<00:00,  9.69it/s]


In [21]:
overall = 0
threshold = 0.1
for id in test_recs:
    score = 0
    for rid in test_recs[id]:
        if len(set([k for k in relevant_id_keywords[rid] if relevant_id_keywords[rid][k] > threshold]).intersection(relevant_id_keywords[id])) > 0:
            score += 1
    
    overall += score / len(test_recs[id])

overall

83.07691843081032

In [27]:
candidate_sets = {id: [] for id in test_recs}

for i in tqdm(range(411)):
    batch = load_json('../support/keyword_batches/{}.json'.format(i))
    for id in test_recs:
        example_keywords = relevant_id_keywords[id]
        for bid in batch:
            if len(set(batch[bid]).intersection(example_keywords)) > 0:
                candidate_sets[id] += [bid]



  0%|          | 0/411 [00:00<?, ?it/s]

  0%|          | 1/411 [00:01<10:01,  1.47s/it]

  0%|          | 2/411 [00:03<10:43,  1.57s/it]

  1%|          | 3/411 [00:05<11:11,  1.65s/it]

  1%|          | 4/411 [00:06<11:23,  1.68s/it]

  1%|          | 5/411 [00:08<11:34,  1.71s/it]

  1%|▏         | 6/411 [00:10<11:43,  1.74s/it]

  2%|▏         | 7/411 [00:12<11:49,  1.75s/it]

  2%|▏         | 8/411 [00:14<11:53,  1.77s/it]

  2%|▏         | 9/411 [00:15<11:52,  1.77s/it]

  2%|▏         | 10/411 [00:17<11:53,  1.78s/it]

  3%|▎         | 11/411 [00:19<11:56,  1.79s/it]

  3%|▎         | 12/411 [00:21<11:59,  1.80s/it]

  3%|▎         | 13/411 [00:22<11:31,  1.74s/it]

  3%|▎         | 14/411 [00:24<11:40,  1.76s/it]

  4%|▎         | 15/411 [00:26<11:46,  1.78s/it]

  4%|▍         | 16/411 [00:28<11:49,  1.80s/it]

  4%|▍         | 17/411 [00:30<11:49,  1.80s/it]

  4%|▍         | 18/411 [00:31<11:50,  1.81s/it]

  5%|▍         | 19/411 [00:33<11:50,  1.81s/it]

  5%|▍         |

 70%|███████   | 288/411 [08:33<03:47,  1.85s/it]

 70%|███████   | 289/411 [08:35<03:46,  1.85s/it]

 71%|███████   | 290/411 [08:36<03:46,  1.87s/it]

 71%|███████   | 291/411 [08:38<03:45,  1.88s/it]

 71%|███████   | 292/411 [08:40<03:45,  1.89s/it]

 71%|███████▏  | 293/411 [08:42<03:44,  1.90s/it]

 72%|███████▏  | 294/411 [08:44<03:39,  1.88s/it]

 72%|███████▏  | 295/411 [08:46<03:36,  1.86s/it]

 72%|███████▏  | 296/411 [08:48<03:36,  1.88s/it]

 72%|███████▏  | 297/411 [08:50<03:36,  1.90s/it]

 73%|███████▎  | 298/411 [08:52<03:34,  1.90s/it]

 73%|███████▎  | 299/411 [08:54<03:33,  1.91s/it]

 73%|███████▎  | 300/411 [08:55<03:32,  1.91s/it]

 73%|███████▎  | 301/411 [08:57<03:31,  1.92s/it]

 73%|███████▎  | 302/411 [08:59<03:30,  1.93s/it]

 74%|███████▎  | 303/411 [09:01<03:28,  1.93s/it]

 74%|███████▍  | 304/411 [09:03<03:22,  1.89s/it]

 74%|███████▍  | 305/411 [09:05<03:21,  1.90s/it]

 74%|███████▍  | 306/411 [09:07<03:17,  1.88s/it]

 75%|███████▍  | 307/411 [09:08

In [28]:
sum([len(candidate_sets[id]) for id in candidate_sets]) / len(candidate_sets)

630626.7291666666

In [29]:
def dump_json(data, output_file_path):
    with open(output_file_path, 'w') as output_file:
        json.dump(data, output_file)

dump_json(candidate_sets, 'filter_by_extracted_keywords.json')

In [30]:
candidate_sets = {id: {} for id in test_recs}

for i in tqdm(range(411)):
    batch = load_json('../support/keyword_batches/{}.json'.format(i))
    for id in test_recs:
        example_keywords = relevant_id_keywords[id]
        for bid in batch:
            intersection = set(batch[bid]).intersection(example_keywords)
            if len(intersection) > 0:
                keyword_importance = 0
                for k in intersection:
                    keyword_importance += batch[bid][k]

                candidate_sets[id][bid] = keyword_importance



  0%|          | 0/411 [00:00<?, ?it/s]

  0%|          | 1/411 [00:01<10:23,  1.52s/it]

  0%|          | 2/411 [00:03<11:06,  1.63s/it]

  1%|          | 3/411 [00:05<11:37,  1.71s/it]

  1%|          | 4/411 [00:07<11:51,  1.75s/it]

  1%|          | 5/411 [00:09<12:04,  1.78s/it]

  1%|▏         | 6/411 [00:10<12:14,  1.81s/it]

  2%|▏         | 7/411 [00:12<12:20,  1.83s/it]

  2%|▏         | 8/411 [00:14<12:28,  1.86s/it]

  2%|▏         | 9/411 [00:16<12:27,  1.86s/it]

  2%|▏         | 10/411 [00:18<12:28,  1.87s/it]

  3%|▎         | 11/411 [00:20<12:32,  1.88s/it]

  3%|▎         | 12/411 [00:22<12:35,  1.89s/it]

  3%|▎         | 13/411 [00:23<12:05,  1.82s/it]

  3%|▎         | 14/411 [00:25<12:14,  1.85s/it]

  4%|▎         | 15/411 [00:27<12:22,  1.88s/it]

  4%|▍         | 16/411 [00:29<12:28,  1.90s/it]

  4%|▍         | 17/411 [00:31<12:29,  1.90s/it]

  4%|▍         | 18/411 [00:33<12:29,  1.91s/it]

  5%|▍         | 19/411 [00:35<12:31,  1.92s/it]

  5%|▍         |

 70%|███████   | 288/411 [09:07<04:04,  1.99s/it]

 70%|███████   | 289/411 [09:09<04:06,  2.02s/it]

 71%|███████   | 290/411 [09:11<04:05,  2.03s/it]

 71%|███████   | 291/411 [09:13<04:05,  2.05s/it]

 71%|███████   | 292/411 [09:15<04:05,  2.06s/it]

 71%|███████▏  | 293/411 [09:17<04:02,  2.06s/it]

 72%|███████▏  | 294/411 [09:20<03:59,  2.05s/it]

 72%|███████▏  | 295/411 [09:21<03:54,  2.02s/it]

 72%|███████▏  | 296/411 [09:24<03:55,  2.05s/it]

 72%|███████▏  | 297/411 [09:26<03:53,  2.05s/it]

 73%|███████▎  | 298/411 [09:28<03:52,  2.06s/it]

 73%|███████▎  | 299/411 [09:30<03:50,  2.06s/it]

 73%|███████▎  | 300/411 [09:32<03:49,  2.07s/it]

 73%|███████▎  | 301/411 [09:34<03:47,  2.07s/it]

 73%|███████▎  | 302/411 [09:36<03:45,  2.07s/it]

 74%|███████▎  | 303/411 [09:38<03:43,  2.07s/it]

 74%|███████▍  | 304/411 [09:40<03:36,  2.03s/it]

 74%|███████▍  | 305/411 [09:42<03:35,  2.03s/it]

 74%|███████▍  | 306/411 [09:44<03:30,  2.01s/it]

 75%|███████▍  | 307/411 [09:46

In [32]:
sorted_candidate_sets = {id: [] for id in candidate_sets}
for id in tqdm(candidate_sets):
    candidate_scores = candidate_sets[id]
    sorted_candidate_sets[id] = sorted(candidate_scores, reverse=True, key=lambda x: candidate_scores[x])



  0%|          | 0/96 [00:00<?, ?it/s]

  1%|          | 1/96 [00:01<02:23,  1.51s/it]

  2%|▏         | 2/96 [00:01<01:51,  1.18s/it]

  3%|▎         | 3/96 [00:02<01:33,  1.00s/it]

  4%|▍         | 4/96 [00:03<01:22,  1.11it/s]

  5%|▌         | 5/96 [00:03<01:18,  1.16it/s]

  6%|▋         | 6/96 [00:05<01:31,  1.01s/it]

  7%|▋         | 7/96 [00:06<01:27,  1.02it/s]

  9%|▉         | 9/96 [00:06<01:09,  1.26it/s]

 10%|█         | 10/96 [00:07<01:11,  1.20it/s]

 11%|█▏        | 11/96 [00:09<01:30,  1.06s/it]

 12%|█▎        | 12/96 [00:11<01:42,  1.22s/it]

 14%|█▎        | 13/96 [00:11<01:34,  1.14s/it]

 15%|█▍        | 14/96 [00:12<01:19,  1.03it/s]

 16%|█▌        | 15/96 [00:13<01:19,  1.02it/s]

 17%|█▋        | 16/96 [00:14<01:16,  1.05it/s]

 18%|█▊        | 17/96 [00:15<01:08,  1.15it/s]

 19%|█▉        | 18/96 [00:15<01:06,  1.18it/s]

 20%|█▉        | 19/96 [00:16<01:04,  1.20it/s]

 21%|██        | 20/96 [00:17<00:51,  1.48it/s]

 22%|██▏       | 21/96 [00:17<00:46

In [40]:
top_k = 100000
overall = 0
for id in sorted_candidate_sets:
    top_candidates = set(sorted_candidate_sets[id][:top_k])
    score = 0
    for cid in test_recs[id]:
        if cid in top_candidates:
            score += 1
    
    overall += score / len(test_recs[id])
overall

66.06550831787455

In [41]:
dump_json(candidate_sets, 'keyword_filtering_candidate_scores.json')

In [42]:
top_k = 50
overall = 0
for id in sorted_candidate_sets:
    top_candidates = set(sorted_candidate_sets[id][:top_k])
    score = 0
    for cid in test_recs[id]:
        if cid in top_candidates:
            score += 1
    
    overall += score / len(test_recs[id])

overall

6.305212254186323